In [ ]:
import __init__

Note for this study its important that the Material ID is in the base df

In [ ]:
import pandas as pd
df = pd.read_parquet('/home/cyprien/Data_Gen/mp-20-data/all_data.parquet')
df

In [ ]:
!python _utils/_preprocessing/_calculate_XRD.py \
    --input_parquet /home/cyprien/Data_Gen/mp-20-data/all_data.parquet \
    --output_parquet HF-databases/mp_20_pxrd/mp_20_pxrd.parquet \
    --num_workers 16

In [ ]:
!python _utils/_preprocessing/_cleaning.py \
    --input_parquet HF-databases/mp_20_pxrd/mp_20_pxrd.parquet \
    --output_parquet HF-databases/mp_20_pxrd/mp_20_pxrd_clean.parquet \
    --property_columns "['Condition Vector']" \
    --num_workers 16 \
    --filter_to 1024

In [ ]:
!python _utils/_preprocessing/_save_dataset_to_HF.py \
    --input_parquet HF-databases/mp_20_pxrd/mp_20_pxrd_clean.parquet \
    --output_parquet HF-databases/mp_20_pxrd/mp_20_pxrd.parquet \
    --save_hub

In [ ]:
!python _utils/_generating/make_prompts.py \
    --HF_dataset 'c-bone/mp_20_pxrd' \
    --split 'test' \
    --automatic \
    --output_parquet '_artifacts/mp-20-pxrd/mp-test_prompts.parquet' \
    --level 'level_3' \
    --condition_columns 'Condition Vector'
    # --remove_ref_columns

# Benchmark with a model trained from scratch

In [ ]:
!python _utils/_generating/generate_CIFs.py --config '_config_files/generation/conditional/xrd_studies/mp-xrd-scratch_eval.jsonc'

In [ ]:
!python _utils/_generating/postprocess.py \
    --input_parquet '_artifacts/mp-20-pxrd/mp-scratch-20perp-test_gen.parquet' \
    --output_parquet '_artifacts/mp-20-pxrd/mp-scratch-20perp-test_post.parquet' \
    --num_workers 32 \
    --column_name 'Generated CIF'

In [ ]:
!python _utils/_metrics/XRD_metrics.py \
    --input_parquet '_artifacts/mp-20-pxrd/mp-scratch-20perp-test_post.parquet' \
    --num_gens 20 \
    --ref_parquet '_artifacts/mp-20-pxrd/mp-test_ref.parquet' \
    --output_parquet '_artifacts/mp-20-pxrd/mp-scratch-20perp-test_metrics.parquet' \
    --num_workers 16 \
    --validity_check 'diffcsp'

In [ ]:
!python _utils/_metrics/XRD_metrics.py \
    --input_parquet '_artifacts/mp-20-pxrd/mp-scratch-20perp-test_post.parquet' \
    --num_gens 1 \
    --ref_parquet '_artifacts/mp-20-pxrd/mp-test_ref.parquet' \
    --output_parquet '_artifacts/mp-20-pxrd/mp-scratch-1perp-nosmact-test_metrics.parquet' \
    --num_workers 24 \
    --validity_check 'none'

In [ ]:
!python _utils/_metrics/XRD_metrics.py \
    --input_parquet '_artifacts/mp-20-pxrd/mp-scratch-20perp-test_post.parquet' \
    --num_gens 1 \
    --ref_parquet '_artifacts/mp-20-pxrd/mp-test_ref.parquet' \
    --output_parquet '_artifacts/mp-20-pxrd/mp-scratch-1rand-test_metrics.parquet' \
    --num_workers 24 \
    --validity_check 'diffcsp' \
    --sort_gens 'random'

In [ ]:
import __init__
import pandas as pd
from _utils import get_metrics_xrd

df_metrics = pd.read_parquet('_artifacts/mp-20-pxrd/mp-scratch-20perp-test_metrics.parquet')
metrics = get_metrics_xrd(df_metrics, n_test=9046, only_matched=True)
print("\n\n")
df_metrics = pd.read_parquet('_artifacts/mp-20-pxrd/mp-scratch-1perp-test_metrics.parquet')
metrics = get_metrics_xrd(df_metrics, n_test=9046, only_matched=True)
print("\n\n")
df_metrics = pd.read_parquet('_artifacts/mp-20-pxrd/mp-scratch-1perp-nosmact-test_metrics.parquet')
metrics = get_metrics_xrd(df_metrics, n_test=9046, only_matched=True)
